In [1]:
from selenium import webdriver
from selenium.webdriver.common import keys
import time
import random
import pytesseract
from pytesseract import image_to_string 
from PIL import Image, ImageEnhance, ImageFilter 


def ParsePic():
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
    im = Image.open("tep.jpg")
    im = im.filter(ImageFilter.CONTOUR)
    im = im.filter(ImageFilter.DETAIL)
    enhancer = ImageEnhance.Contrast(im)
    im = enhancer.enhance(4)
    im = im.convert('L')
    im.save('temp10.png')  
    text = image_to_string(Image.open('tep.jpg'))
    print(text)

In [2]:
ParsePic()

In [3]:
from bs4 import BeautifulSoup
import requests
import bs4
import urllib.request
import pandas as pd
from pytesseract import image_to_string 
from PIL import Image
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


def get_captcha_text(location, size):
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
    im = Image.open('screen.png') # uses PIL library to open image in memory

    left = location['x']
    top = location['y']
    right = location['x'] + size['width']
    bottom = location['y'] + size['height']


    im = im.crop((left, top, right, bottom)) # defines crop points
    im.save('screen.png')
    captcha_text = image_to_string(Image.open('screen.png'))
    return captcha_text

def pdfdownload(url,filename):
    import urllib.request
    pdf_path =url
    response = urllib.request.urlopen(url)    
    file = open(r"E:/pdffiles/PdfDocument Scarping2/"+filename+".pdf", 'wb')
    file.write(response.read())
    
    file.close()

def login_to_website(FromDate,ToDate):
    url = "https://services.ecourts.gov.in/ecourtindiaHC/cases/s_orderdate.php?state_cd=15&dist_cd=1&court_code=1&stateNm=Uttarakhand"
    driver = webdriver.Chrome(executable_path = "G:/chromedriver/chromedriver.exe")
    driver.get(url)
    #driver.set_window_size(1120, 550)
    element = driver.find_element_by_xpath('/html/body/form/div[2]/div[3]/span/div[1]/div/span[3]/img') # find part of the page you want image of
    location = element.location
    size = element.size
    driver.save_screenshot('screen.png')
    
    driver.find_element_by_id("to_date").send_keys(ToDate)
    driver.find_element_by_id("from_date").send_keys(FromDate)
    captcha = driver.find_element_by_xpath('//*[@id="captcha_image"]')
    #captcha.clear()
    captcha_text = get_captcha_text(location, size)
    captcha.send_keys(captcha_text)
    print(captcha_text)
    driver.find_element_by_id("captcha").send_keys(captcha_text)
    sleep(5)
    driver.find_element_by_xpath("/html/body/form/div[2]/div[4]/span[3]/input[1]").click()
    
    
    
    sleep(5)
    page_source = driver.page_source
    soup1 = BeautifulSoup(page_source, 'lxml')
    type(soup1)
    bs4.BeautifulSoup
    title = soup1.title
    case=[]
    CaseType_CaseNumber_CaseYear=[]
    Order_Date=[]
    Order_Number=[]
    status=[]
    
    
    


    PDF_file_Table_tag=soup1.find_all("table",{"id":"showList3"})[0]

    tr_tag=PDF_file_Table_tag.find_all("tr")
    print(len(tr_tag))

    for i in range(2,len(tr_tag)):
        td_tag=tr_tag[i].find_all("td") [0] 
        td_tag1=tr_tag[i].find_all("td")[1]
        td_tag2=tr_tag[i].find_all("td")[2]
        td_tag3=tr_tag[i].find_all("td")[3]
        

        case.append(td_tag.text)
        CaseType_CaseNumber_CaseYear.append(td_tag1.text)
        Order_Date.append(td_tag2.text)
        Order_Number.append(td_tag3.text)
        
        
        document=td_tag1.text
        print(document)
        try:
          
            a_tag_url=td_tag3.find("a")["href"]
            status.append(a_tag_url)
            print(str(i-1)+")"+a_tag_url)
            pdfdownload("https://services.ecourts.gov.in/ecourtindiaHC/cases/"+ a_tag_url,document.replace("/"," "))
            
        except:
            Order_Number.append("NaN")
            
            print("No Document")
    a={"case":case,"CaseType_CaseNumber_CaseYear":CaseType_CaseNumber_CaseYear,"Order_Date":Order_Date,"Order_Number":Order_Number,"statulink":status}
            
    df = pd.DataFrame.from_dict(a, orient='index')
    df = df.transpose()
    df.to_csv(r"E:\pdffiles\csvfiles\courtt.csv", sep=',',index=False)
    df.head(50)

    
    
  
      
login_to_website("11-02-2021","12-02-2021")


83906


200
VATR/2/2021
1)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoPX6Nsx3sBKjR0rvO97j2aQe%2BDLsT%2FlDra85xvtnQiDf&caseno=VATR/2/2021&cCode=1&appFlag=
CTR/1/2021
2)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoBlGpga8qdNfYkzz%2Bey3HpGMElCRsJe2J5%2BUIOWRS5qQ&caseno=CTR/1/2021&cCode=1&appFlag=
CTR/2/2021
3)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoKxbbnH3qK30ZbaLvfy2sJsZte341qRMHMSGPpUXoCuS&caseno=CTR/2/2021&cCode=1&appFlag=
CTR/3/2021
4)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoG3DLK9l8oCRPTh7B3%2Fzoh188WZcdyUz8ON9lqBj1LO0&caseno=CTR/3/2021&cCode=1&appFlag=
CTR/4/2021
5)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoIPPoH3OM39Df%2BIfjJJfZzOHW9P4S22sRoqAI%2FLe6i%2Bo&caseno=CTR/4/2021&cCode=1&appFlag=
BA1/32/2021
6)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoNP3PuO5tiC%2FfbldT9xuUcm6iiMGmwM8xwUVKtK31Zz3&caseno=BA1/32/2021&cCode=1&appFlag=
BA1/57/2021
7)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoNYHvjCw0OIwMAQJEadRipYzWJAyqNptG2Zutkz%2Fa2F8&caseno=BA1/57/2021

WPMS/1093/2019
57)display_pdf.php?filename=%2FE3WiyNUWFIaR1oBGE62Wl9NVE42TediB07GafznfWbD9W5n%2F5Yub8%2B1hgU3S0PX&caseno=WPMS/1093/2019&cCode=1&appFlag=
WPMS/1445/2019
58)display_pdf.php?filename=%2FE3WiyNUWFIaR1oBGE62WgFThBsNZIpqv%2BeqGM3Aiv1vHSyWo17l6CtgS%2FWR4l4p&caseno=WPMS/1445/2019&cCode=1&appFlag=
WPMS/1500/2016
59)display_pdf.php?filename=7yg5D%2FmJmLJFbv9l4Wl3vauh6VUsRbFeURZDYvH%2FS%2F%2By7LJKJ30GwK1JuGWTnyLb&caseno=WPMS/1500/2016&cCode=1&appFlag=
WPMS/2074/2016
60)display_pdf.php?filename=7yg5D%2FmJmLJFbv9l4Wl3vej%2F7NfURTeSL796tuLY%2Bg%2FBUYRVeqglvSEsGmbNp7jj&caseno=WPMS/2074/2016&cCode=1&appFlag=
WPMS/2396/2019
61)display_pdf.php?filename=%2FE3WiyNUWFIaR1oBGE62WguEmbN9eZq6I1yHhhvobPYRxJHhjYkfJJhsugovjtCl&caseno=WPMS/2396/2019&cCode=1&appFlag=
WPMS/3084/2016
62)display_pdf.php?filename=7yg5D%2FmJmLJFbv9l4Wl3vRH6oA4mCt6DkHQvfurEsVjDNztC%2FJmHJPibS5JHbAXH&caseno=WPMS/3084/2016&cCode=1&appFlag=
WPMS/3603/2019
63)display_pdf.php?filename=%2FE3WiyNUWFIaR1oBGE62WpFYcx2Y0bX4%2BzVKu

BA1/330/2021
112)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoHehvAFtpR1ndwSCNKNNUUpoXwCCq6PMl78Q3nc0avsc&caseno=BA1/330/2021&cCode=1&appFlag=
BA1/335/2021
113)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoAdOjyEgxVCCBd5udo9n3RSyIeg9udVxN2Nv9pUQkgg5&caseno=BA1/335/2021&cCode=1&appFlag=
BA1/336/2021
114)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoNTWDp3kvYB%2FLEtyZCqLLIV4wJD1vx7NaPRzTd%2B7DScu&caseno=BA1/336/2021&cCode=1&appFlag=
BA1/337/2021
115)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoNlA9SKEp%2FZdxY6xlx3WOTBPDu4gOOp44577eA3a8B2d&caseno=BA1/337/2021&cCode=1&appFlag=
BA1/338/2021
116)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoOm4%2BhMzxuOZ0aylFNBCPIwWRcPFbz4UUyzy0xIVXFS6&caseno=BA1/338/2021&cCode=1&appFlag=
BA1/339/2021
117)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoGgMpnja1S6UCr1UNGUg2D2cKpoSddYoZdHoTXZHdu8o&caseno=BA1/339/2021&cCode=1&appFlag=
BA1/340/2021
118)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoFtJ9eLX3x778u7kKHUa70r%2FMPNX%2FJVesuJCTv4bz7z%2F&c

C482/238/2021
168)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoGM%2F8mRqwBq4uKsKsIKuFm0JkevoMVBLhShqoIfhcaI%2B&caseno=C482/238/2021&cCode=1&appFlag=
C482/240/2021
169)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoP4kQsV6O7NUeRAnVi4hsAGJITN29c2wsbFQnND7k2NM&caseno=C482/240/2021&cCode=1&appFlag=
C482/241/2021
170)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoEBpBlBc0W9uAKeFGOm%2FgVhtU0THf9O4iKJcoLLgK%2FnG&caseno=C482/241/2021&cCode=1&appFlag=
C482/242/2021
171)display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoJAVxRg4I7swieTcaWDW6LDNDSlKfX0XSGj8yELGl2oX&caseno=C482/242/2021&cCode=1&appFlag=
C482/439/2020
172)display_pdf.php?filename=U%2BbhtlrLe2adAHN8Tz%2F1d%2FWMx8fhT2dcyHRwXPH69sV6aQpnyZLk5QP3OoExhFzA&caseno=C482/439/2020&cCode=1&appFlag=
C482/841/2020
173)display_pdf.php?filename=U%2BbhtlrLe2adAHN8Tz%2F1d486sO7hO4Jhol9zDeZ8vGmeHg9ky8ltnc8yUT8LZD%2Fx&caseno=C482/841/2020&cCode=1&appFlag=
C482/1249/2014
174)display_pdf.php?filename=lG6h6ilt3H1fOBr4DLdM9QPgOpYPZT%2BevkRo6uohnA69MOyFfV

QnBUxJ6a3gIx%2B5SFrUiAoJgmvGJIGROR7Spzcr1uQCU3q9fFfnMbWOLtH9ID28Xz&caseno=WPCRL/275/2021&cCode=1&appFlag=

display_pdf.php?filename=QnBUxJ6a3gIx%2B5SFrUiAoJgmvGJIGROR7Spzcr1uQCU3q9fFfnMbWOLtH9ID28Xz&caseno=WPCRL/275/2021.pdf